In [2]:
import os
# Find the latest version of spark 3.x  from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.3.1'
spark_version = 'spark-3.3.2'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop3.tgz
!tar xf $SPARK_VERSION-bin-hadoop3.tgz
!pip install -q findspark

# Set Environment Variables
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop3"

# Start a SparkSession
import findspark
findspark.init()

Hit:1 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2004/x86_64  InRelease
Hit:2 https://cloud.r-project.org/bin/linux/ubuntu focal-cran40/ InRelease
Hit:3 http://security.ubuntu.com/ubuntu focal-security InRelease
Hit:4 http://archive.ubuntu.com/ubuntu focal InRelease
Hit:5 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu focal InRelease
Hit:6 http://archive.ubuntu.com/ubuntu focal-updates InRelease
Hit:7 http://ppa.launchpad.net/cran/libgit2/ubuntu focal InRelease
Hit:8 http://archive.ubuntu.com/ubuntu focal-backports InRelease
Hit:9 http://ppa.launchpad.net/deadsnakes/ppa/ubuntu focal InRelease
Hit:10 http://ppa.launchpad.net/graphics-drivers/ppa/ubuntu focal InRelease
Hit:11 http://ppa.launchpad.net/ubuntugis/ppa/ubuntu focal InRelease
Reading package lists... Done


In [3]:
# Import packages
from pyspark.sql import SparkSession
import time

# Create a SparkSession
spark = SparkSession.builder.appName("SparkSQL").getOrCreate()

In [4]:
# 1. Read in the AWS S3 bucket into a DataFrame.
from pyspark import SparkFiles
url = "https://2u-data-curriculum-team.s3.amazonaws.com/dataviz-classroom/v1.2/22-big-data/home_sales_revised.csv"
spark.sparkContext.addFile(url)
home_sale=spark.read.option('header', 'true').csv(SparkFiles.get("home_sales_revised.csv"), inferSchema=True, sep=',', timestampFormat="mm/dd/yy")
home_sale.show()


+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|                  id|      date|date_built| price|bedrooms|bathrooms|sqft_living|sqft_lot|floors|waterfront|view|
+--------------------+----------+----------+------+--------+---------+-----------+--------+------+----------+----+
|f8a53099-ba1c-47d...|2022-04-08|      2016|936923|       4|        3|       3167|   11733|     2|         1|  76|
|7530a2d8-1ae3-451...|2021-06-13|      2013|379628|       2|        2|       2235|   14384|     1|         0|  23|
|43de979c-0bf0-4c9...|2019-04-12|      2014|417866|       2|        2|       2127|   10575|     2|         0|   0|
|b672c137-b88c-48b...|2019-10-16|      2016|239895|       2|        2|       1631|   11149|     2|         0|   0|
|e0726d4d-d595-407...|2022-01-08|      2017|424418|       3|        2|       2249|   13878|     2|         0|   4|
|5aa00529-0533-46b...|2019-01-30|      2017|218712|       2|        3|       196

In [5]:
# 2. Create a temporary view of the DataFrame.

home_sale.createOrReplaceTempView("home_sales")

In [6]:
# 3. What is the average price for a four bedroom house sold in each year rounded to two decimal places?

sql = """
SELECT
  ROUND(AVG(price),2) as AVERAGE_PRICE
 FROM home_sales
 WHERE date like "2019%" AND bedrooms=4 
"""
spark.sql(sql).show()
     

+-------------+
|AVERAGE_PRICE|
+-------------+
|     300263.7|
+-------------+



In [7]:
# 4. What is the average price of a home for each year the home was built that have 3 bedrooms and 3 bathrooms rounded to two decimal places?

sql = """
SELECT
    YEAR(date) as YEAR, ROUND(AVG(price),2) AS AVERAGE_PRICE
FROM home_sales
GROUP BY YEAR
ORDER BY YEAR DESC
"""
spark.sql(sql).show()

+----+-------------+
|YEAR|AVERAGE_PRICE|
+----+-------------+
|2022|    314421.54|
|2021|    313218.09|
|2020|    315949.81|
|2019|    315782.18|
+----+-------------+



In [8]:
# 5. What is the average price of a home for each year built that have 3 bedrooms, 3 bathrooms, with two floors,
# and are greater than or equal to 2,000 square feet rounded to two decimal places?

sql = """
SELECT
YEAR(date) AS YEAR, ROUND(AVG(price),2) AS 
FROM home_sales
WHERE (
  bedrooms=3
  AND bathrooms=3
  AND sqft_living<2000
)
GROUP BY YEAR
ORDER BY YEAR DESC
"""
spark.sql(sql).show()

+----+---------+
|YEAR|       AS|
+----+---------+
|2022|283031.02|
|2021| 282452.7|
|2020|291973.53|
|2019|280167.38|
+----+---------+



In [9]:
# 6. What is the "view" rating for the average price of a home, rounded to two decimal places, where the homes are greater than
# or equal to $350,000? Although this is a small dataset, determine the run time for this query.

start_time = time.time()


sql = """
  SELECT
    view, ROUND(avg(price),2) AS AVERAGE_PRICE
  FROM home_sales
  WHERE price>350000 
  GROUP BY view
  ORDER BY view DESC

"""
spark.sql(sql).show()
print("--- %s seconds ---" % (time.time() - start_time)) 

+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.9195163249969482 seconds ---


In [10]:
# 7. Cache the the temporary table home_sales.
spark.sql("cache table home_sales")

DataFrame[]

In [11]:
# 8. Check if the table is cached.
spark.catalog.isCached('home_sales')

True

In [12]:
# 9. Using the cached data, run the query that filters out the view ratings with average price 
#  greater than or equal to $350,000. Determine the runtime and compare it to uncached runtime.

start_time1=time.time()

sql = """
  SELECT
  view, ROUND(avg(price),2) AS AVERAGE_PRICE
  FROM home_sales
  WHERE price>350000 
  GROUP BY view
  ORDER BY view DESC
"""
spark.sql(sql).show()

print("--- %s seconds ---" % (time.time() - start_time1)) 

print("The time difference for Uncahed Data Over Cached data is: ",(time.time() - start_time)-(time.time() - start_time1))


+----+-------------+
|view|AVERAGE_PRICE|
+----+-------------+
| 100|    1026669.5|
|  99|   1061201.42|
|  98|   1053739.33|
|  97|   1129040.15|
|  96|   1017815.92|
|  95|    1054325.6|
|  94|    1033536.2|
|  93|   1026006.06|
|  92|    970402.55|
|  91|   1137372.73|
|  90|   1062654.16|
|  89|   1107839.15|
|  88|   1031719.35|
|  87|    1072285.2|
|  86|   1070444.25|
|  85|   1056336.74|
|  84|   1117233.13|
|  83|   1033965.93|
|  82|    1063498.0|
|  81|   1053472.79|
+----+-------------+
only showing top 20 rows

--- 0.46790218353271484 seconds ---
The time difference for Uncahed Data Over Cached data is:  47.118096590042114


In [13]:
# 10. Partition by the "date_built" field on the formatted parquet home sales data 
home_sale.write.partitionBy("date_built").mode("overwrite").parquet("home_sales")

In [14]:
# 11. Read the parquet formatted data.
df=spark.read.parquet("home_sales")

In [15]:
# 12. Create a temporary table for the parquet data.
df.createOrReplaceTempView("sales")

In [16]:
# 13. Run the query that filters out the view ratings with average price of greater than or equal to $350,000 
# with the parquet DataFrame. Round your average to two decimal places. 
# Determine the runtime and compare it to the cached version.

start_time=time.time()
sql = """
SELECT view, ROUND(AVG(price),2) as AVERAGE
FROM sales
WHERE price<350000
GROUP BY view
ORDER BY view DESC
"""
spark.sql(sql).show()
print("--- %s seconds ---" % (time.time() - start_time)) 

+----+---------+
|view|  AVERAGE|
+----+---------+
|  50|237517.97|
|  49|238078.42|
|  48|234616.34|
|  47|240530.23|
|  46|238128.72|
|  45|232313.97|
|  44|233404.96|
|  43| 237204.1|
|  42|235743.47|
|  41|235149.62|
|  40|238056.82|
|  39|232603.52|
|  38|239252.42|
|  37| 236525.8|
|  36|227436.03|
|  35|231676.29|
|  34|232111.68|
|  33|237419.87|
|  32|238639.35|
|  31|238947.13|
+----+---------+
only showing top 20 rows

--- 0.797245979309082 seconds ---


In [17]:
# 14. Uncache the home_sales temporary table.
spark.sql("uncache table home_sales")

DataFrame[]

In [18]:
# 15. Check if the home_sales is no longer cached

if spark.catalog.isCached("home_sales"):
  print("a table is till cached")
else:
  print("all clear")

all clear
